In [1]:
# -------------------------

# This notebook includes the "Spatial Segment Deficiency" tool

# -------------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import warnings
warnings.simplefilter(action='ignore')
from tqdm import tqdm
from pathlib import Path
import seaborn as sns
import zipfile
import geopandas as gpd
from shapely.geometry import Point, LineString
from scipy import stats
import datetime
import matplotlib.dates as mdates
import plotly.graph_objects as go
import folium
from matplotlib.ticker import MaxNLocator
import shapely
from folium.plugins import GroupedLayerControl, FeatureGroupSubGroup
from branca.element import Element


In [2]:
season_queries = {
    "2023-1": "service_date >= '2022-12-18' & service_date <= '2023-03-11'",
    "2023-2": "service_date >= '2023-03-12' & service_date <= '2023-07-02'",
    "2023-3": "service_date > '2023-07-02' & service_date <= '2023-08-26'",
    "2023-4"  : "service_date >= '2023-08-27' & service_date <= '2023-12-16'",
    "2024-1": "service_date >= '2023-12-17' & service_date <= '2024-04-06'",
    "2024-2": "service_date >= '2024-04-07' & service_date <= '2024-06-15'",
    "2024-3": "service_date >= '2024-06-16' & service_date <= '2024-08-24'",
    "2024-4"  : "service_date > '2024-08-24' & service_date <= '2024-12-14'",
    "2025-1": "service_date > '2024-12-14' & service_date <= '2025-04-06'",
    "2025-2": "service_date > '2025-04-06' & service_date <= '2025-06-14'",
}

In [3]:
# -------------------------

# Read pre-calculated dataframes from data_process.ipynb
# df_all: cleaned AVL data
# stops: bus stop coordinates from the latest rating

# -------------------------


df_all = pd.read_csv('df_all.csv')

df_all["service_date"]     = pd.to_datetime(df_all["service_date"],     format="%Y-%m-%d",              errors="coerce")
df_all["scheduled_boston"] = pd.to_datetime(df_all["scheduled_boston"], format="%Y-%m-%d %H:%M:%S",     errors="coerce")
df_all["actual_boston"]    = pd.to_datetime(df_all["actual_boston"],    format="%Y-%m-%d %H:%M:%S",     errors="coerce")

all_available_routes = df_all.drop_duplicates('route_id')
all_available_routes = all_available_routes[all_available_routes['route_id'].astype(str).str.strip().str.fullmatch(r'\d+')]


stops = pd.read_excel('Stops-2-2025.xlsx')
stops_all=stops.drop_duplicates('Place')[['Place','Lat','Long']].dropna()

In [4]:
# -------------------------

# Some stop names include capital letters, normalize both dataframes to avoid confussion

# -------------------------
df_all = df_all.rename(columns={'timepointid':'time_point_id'})

df_all['time_point_id'] = (
    df_all['time_point_id']
      .astype(str)               
      .str.lower()                
      .str.replace(r'\s+', '', regex=True)  
)

stops_all['Place'] = (
    stops_all['Place']
      .astype(str)               
      .str.lower()                
      .str.replace(r'\s+', '', regex=True)  
)


In [5]:
# -------------------------

# Read the latest MBTA GTFS for bus route shapefiles

# -------------------------

feed = "MBTA_GTFS.zip"
with zipfile.ZipFile(feed) as z:
    routes      = pd.read_csv(z.open("routes.txt"))
    trips       = pd.read_csv(z.open("trips.txt"))
    stop_times  = pd.read_csv(z.open("stop_times.txt"))
    stops       = pd.read_csv(z.open("stops.txt"))
    shapes      = pd.read_csv(z.open("shapes.txt"))


In [6]:
shapes_sorted = (shapes
                 .sort_values(["shape_id", "shape_pt_sequence"])
                 .loc[:, ["shape_id", "shape_pt_lon", "shape_pt_lat"]])

geometry_by_shape = (shapes_sorted
                     .groupby("shape_id")        
                     .apply(lambda df: LineString(zip(df.shape_pt_lon,
                                                     df.shape_pt_lat)))
)

shape_gdf = gpd.GeoDataFrame(
    geometry_by_shape.rename("geometry"),
    crs="EPSG:4326"      
).reset_index()            
shape_route_map = (trips[["route_id", "shape_id"]]
                   .drop_duplicates())

# choose the most-frequent shape per route
canonical = (trips.value_counts(["route_id", "shape_id"])
                    .rename("n_trips")
                    .reset_index()
                    .sort_values(["route_id", "n_trips"], ascending=[True, False])
                    .drop_duplicates("route_id"))[["route_id", "shape_id"]]

shape_route_map = canonical

route_gdf = (shape_route_map
             .merge(shape_gdf, on="shape_id", how="left")
             .merge(routes[["route_id", "route_long_name",
                            "route_short_name", "route_type"]],
                    on="route_id", how="left"))

route_gdf = gpd.GeoDataFrame(route_gdf, geometry="geometry", crs="EPSG:4326")


In [7]:
def _norm_rt(x):
    try:
        return f"{int(x):02d}"
    except (ValueError, TypeError):
        return str(x)

route_geoms = {}
for rid, geom in zip(route_gdf['route_id'], route_gdf['geometry']):
    k = _norm_rt(rid)
    route_geoms[k] = geom


In [8]:
MIN_ROUTES =2   # Minimum number of busses running for a segment to be classified as a shared segment


def _add_polyline_with_border(group, coords, *, color, weight,
                              border_color="black",
                              border_pad=1,         
                              border_opacity=0.35,  
                              **kwargs):

    border_kwargs = {k: v for k, v in kwargs.items()
                     if k not in ("opacity", "tooltip")}
    folium.PolyLine(
        coords,
        color=border_color,
        weight=weight + border_pad,
        opacity=border_opacity,
        **border_kwargs
    ).add_to(group)

    folium.PolyLine(
        coords,
        color=color,
        weight=weight,
        **kwargs
    ).add_to(group)


def segment_diffs(df, season, time_col):
    """
    Compute per-trip segment runtimes
    """
    out = []
    df = df.copy()
    for k in sorted(df['time_point_order'].unique()):
        end = (df[df['time_point_order'] == k + 1]
               [['half_trip_id', time_col, 'time_point_id', 'route_id', 'direction_id']]
               .rename(columns={time_col: 't_end', 'time_point_id': 'time_point_order_end'}))
        start = (df[df['time_point_order'] == k]
                 [['half_trip_id', time_col, 'hour', 'time_point_id', 'route_id', 'direction_id']]
                 .rename(columns={time_col: 't_start'}))
        merged = start.merge(end, on=['half_trip_id', 'route_id', 'direction_id'], how='inner')
        merged['k'] = k
        merged['time_diff'] = (merged['t_end'] - merged['t_start']).dt.total_seconds() / 60
        out.append(merged[['half_trip_id','route_id','direction_id','k','hour',
                           'time_diff','time_point_id','time_point_order_end']])
    seg = pd.concat(out, ignore_index=True)
    seg['season'] = season
    seg['kind']   = 'actual' if 'actual' in time_col else 'scheduled'
    seg = seg[seg['time_diff'] > 0]
    return seg

def build_mapit_for_season_corridor(df_all, season_queries, stops_all, season,
                                    routes, direction, DOW):

    base = df_all.query(season_queries[season]).copy()

    base = base[(base['DOW'] == DOW) &
                (base['direction_id'] == direction) &
                (base['route_id'].isin(routes))].copy()
    
    # Nubn and dudly tags are used for the same stop
    base['time_point_id'] = base['time_point_id'].astype(str).str.replace(
        r'^\s*nubn\s*$', 'dudly', case=False, regex=True
    )
    # compute per-trip segment runtimes
    actual = segment_diffs(base, season, 'actual_boston')
    sched  = segment_diffs(base, season, 'scheduled_boston')

    # keep "typical" trips per route (median segment count by half_trip_id within each route)
    cnts = (actual.groupby(['route_id','half_trip_id'])
                  .size()
                  .reset_index(name='count'))
    med_by_route = cnts.groupby('route_id')['count'].median().to_dict()
    keep = cnts[cnts.apply(lambda r: r['count'] == med_by_route.get(r['route_id'], r['count']), axis=1)]
    keep_key = set(zip(keep['route_id'], keep['half_trip_id']))

    actual = actual[actual.apply(lambda r: (r['route_id'], r['half_trip_id']) in keep_key, axis=1)]
    sched  = sched [sched .apply(lambda r: (r['route_id'], r['half_trip_id']) in keep_key, axis=1)]


    # add start/end coords from stops table
    actual = actual.merge(stops_all, left_on='time_point_id',       right_on='Place', how='left')
    actual = actual.merge(stops_all, left_on='time_point_order_end', right_on='Place', how='left',
                          suffixes=('_start','_end'))
    actual = actual.rename(columns={
        'Lat_start':'Lat_x', 'Long_start':'Long_x',
        'Lat_end':'Lat_y',   'Long_end':'Long_y'
    })

    # match actual vs scheduled (per trip/segment/route)
    mapit = actual.merge(
        sched[['half_trip_id','route_id','direction_id','k','hour','time_diff']],
        on=['half_trip_id','route_id','direction_id','k','hour'],
        how='inner',
        suffixes=('_x','_y')  # -> time_diff_x (actual) & time_diff_y (scheduled)
    )
    mapit['season'] = season
    

    final_cnts = mapit.value_counts(['route_id','time_point_id']).reset_index()
    p = 0.80 
    thr = final_cnts.groupby('route_id')['count'].transform(lambda s: p * s.max())
    final_cnts_filtered = final_cnts[final_cnts['count'] >= thr].copy()
    
    final_cnts_filtered['key'] = final_cnts_filtered['route_id'].astype(str) +'-' +final_cnts_filtered['time_point_id'].astype(str)
    mapit['key'] = mapit['route_id'].astype(str) +'-' +mapit['time_point_id'].astype(str)

    mapit = mapit[mapit['key'].isin(final_cnts_filtered['key'])]
    
    return mapit


def _corridor_layers_with_route_counts(
    season_mapit, hour, k_first, k_last, cmap, season_label,
    min_routes=MIN_ROUTES, weight_base=2, weight_step=2,
    route_geoms=None
):
    """
    Produces three overlays for a given season/hour/k-range:
      1: Shared segments (≥ min_routes)
      2: All segments (thickness ∝ number of routes)
      3: One overlay per route (route picker)
    """
    from shapely.geometry import Point as _Point, LineString as _LineString
    try:
        from shapely.ops import substring as _substring
    except Exception:
        _substring = None  

    # columns we need from the season slice
    need = [
        'hour','k','time_diff_x','time_diff_y','Lat_x','Long_x','Lat_y','Long_y',
        'time_point_id','time_point_order_end','route_id'
    ]
    sub = season_mapit.loc[
        (_in_hour_range(season_mapit['hour'], hour)) &
        (season_mapit['k'].between(k_first, k_last)),
        need
    ].copy()
    if sub.empty or len(sub)<30:
        return None, None, [], None

    # keep valid sched times & coords
    sub = sub[sub['time_diff_y'] > 0].copy()
    if sub.empty:
        return None, None, [], None

    for c in ['Lat_x','Long_x','Lat_y','Long_y']:
        sub[c] = pd.to_numeric(sub[c], errors='coerce')
    sub = sub.dropna(subset=['Lat_x','Long_x','Lat_y','Long_y']).copy()
    if sub.empty:
        return None, None, [], None

    # relative delta per row
    sub['delta'] = (sub['time_diff_x'] - sub['time_diff_y']) / sub['time_diff_y']
    
    # 1) mean within each route per segment (routes equally weighted)
    per_route = (
        sub.groupby(['route_id','time_point_id','time_point_order_end'], as_index=False)
           .agg(delta=('delta','mean'),
                act_mean=('time_diff_x','mean'),
                sch_mean=('time_diff_y','mean'),
                Lat_x=('Lat_x','mean'),
                Long_x=('Long_x','mean'),
                Lat_y=('Lat_y','mean'),
                Long_y=('Long_y','mean'))
    )

    # 2) aggregate across routes to segment-level, including route list
    def _routes_agg(x):
        return tuple(sorted(map(str, pd.Series(x).unique())))

    seg_agg = (
        per_route.groupby(['time_point_id','time_point_order_end'], as_index=False)
                 .agg(delta=('delta','mean'),
                      act_mean=('act_mean','mean'),
                      sch_mean=('sch_mean','mean'),
                      Lat_x=('Lat_x','mean'),
                      Long_x=('Long_x','mean'),
                      Lat_y=('Lat_y','mean'),
                      Long_y=('Long_y','mean'),
                      routes=('route_id', _routes_agg))
    )
    seg_agg['n_routes'] = seg_agg['routes'].apply(len)

    seg_agg.replace([np.inf, -np.inf], np.nan, inplace=True)
    seg_agg = seg_agg.dropna(subset=['Lat_x','Long_x','Lat_y','Long_y','delta','act_mean','sch_mean'])
    if seg_agg.empty:
        return None, None, [], None

    label_hr = _hour_label(hour)
    
    def _slice_route_segment(line, p_start_latlon, p_end_latlon):
        """
        Slice the LineString between nearest points to p_start/p_end.
        Input points are (lat, lon); convert to (lon, lat) for Shapely.
        Returns coords as [(lat, lon), ...] for Folium.
        """
        if not isinstance(line, _LineString) or _substring is None:
            return [p_start_latlon, p_end_latlon]  

        p_start = _Point(p_start_latlon[1], p_start_latlon[0])
        p_end   = _Point(p_end_latlon[1],   p_end_latlon[0])

        try:
            d1, d2 = line.project(p_start), line.project(p_end)
            if d1 == d2:
                return [p_start_latlon, p_end_latlon]
            if d1 > d2:
                d1, d2 = d2, d1
            seg = _substring(line, d1, d2)
            if seg.is_empty:
                return [p_start_latlon, p_end_latlon]
            coords_ll = [(y, x) for (x, y) in seg.coords]  

            if _Point(coords_ll[0][1], coords_ll[0][0]).distance(p_start) > \
               _Point(coords_ll[-1][1], coords_ll[-1][0]).distance(p_start):
                coords_ll = list(reversed(coords_ll))
            return coords_ll
        except Exception:
            return [p_start_latlon, p_end_latlon]

    # ---------- A) Shared segments (≥ min_routes) ----------
    fg_shared = folium.FeatureGroup(
        name=f"{season_label} • {label_hr} • Shared (≥{min_routes})",
        overlay=True, control=True, show=False
    )
    for _, r in seg_agg[seg_agg['n_routes'] >= int(min_routes)].iterrows():
        start_latlon = (float(r['Lat_x']), float(r['Long_x']))
        end_latlon   = (float(r['Lat_y']), float(r['Long_y']))
        d            = float(r['delta'])
        nr           = int(r['n_routes'])
        routes_str   = ", ".join(map(str, r['routes']))
        start_id     = str(r['time_point_id'])
        end_id       = str(r['time_point_order_end'])

        # choose a representative route’s geometry (first in the list)
        coords = [start_latlon, end_latlon]
        if route_geoms:
            rt_choice = str(r['routes'][0]) if len(r['routes']) else None
            line_full = route_geoms.get(rt_choice)
            if isinstance(line_full, _LineString):
                coords = _slice_route_segment(line_full, start_latlon, end_latlon)

        _add_polyline_with_border(
            fg_shared,coords,
            color=cmap(d),
            weight=6,
            opacity=0.9,
            tooltip=(f"<div style='font-size:14px; font-weight:bold'>"
                f"{season_label} • {label_hr}"
                     f"<br>start_id: {start_id} → end_id: {end_id}"
                     f"<br>n_routes={nr} • Δ̄={d:.2f}"
                     f"<br>(act̄={r['act_mean']:.2f}, sch̄={r['sch_mean']:.2f})"
                     f"<br>routes: {routes_str}")
        )

    # ---------- B) All segments (thickness ∝ n_routes) ----------
    fg_all = folium.FeatureGroup(
        name=f"{season_label} • {label_hr} • All segments (thickness∝routes)",
        overlay=True, control=True, show=False
    )
    for _, r in seg_agg.iterrows():
        start_latlon = (float(r['Lat_x']), float(r['Long_x']))
        end_latlon   = (float(r['Lat_y']), float(r['Long_y']))
        d            = float(r['delta'])
        nr           = int(r['n_routes'])
        routes_str   = ", ".join(map(str, r['routes']))
        start_id     = str(r['time_point_id'])
        end_id       = str(r['time_point_order_end'])

        w = weight_base + (nr - 1) * weight_step

        coords = [start_latlon, end_latlon]
        if route_geoms:
            rt_choice = str(r['routes'][0]) if len(r['routes']) else None
            line_full = route_geoms.get(rt_choice)
            if isinstance(line_full, _LineString):
                coords = _slice_route_segment(line_full, start_latlon, end_latlon)

        _add_polyline_with_border(fg_all,
            coords,
            color=cmap(d),
            weight=w,
            opacity=0.9,
            tooltip=(f"<div style='font-size:14px; font-weight:bold'>"
                f"{season_label} • {label_hr}"
                     f"<br>start_id: {start_id} → end_id: {end_id}"
                     f"<br>n_routes={nr} • Δ̄={d:.2f}"
                     f"<br>(act̄={r['act_mean']:.2f}, sch̄={r['sch_mean']:.2f})"
                     f"<br>routes: {routes_str}")
        )
    # ---------- C) Per-route overlays ----------
    route_groups = []
    for rt, g in per_route.groupby('route_id'):
        fg_rt = folium.FeatureGroup(
            name=f"{season_label} • Route {rt}", overlay=True, control=True, show=False
        )
        # obtain the full route geometry once
        line_full = route_geoms.get(str(rt)) if route_geoms else None
        for _, r in g.iterrows():
            start_latlon = (float(r['Lat_x']), float(r['Long_x']))
            end_latlon   = (float(r['Lat_y']), float(r['Long_y']))
            d            = float(r['delta'])
            start_id     = str(r['time_point_id'])
            end_id       = str(r['time_point_order_end'])

            if isinstance(line_full, _LineString):
                coords = _slice_route_segment(line_full, start_latlon, end_latlon)
            else:
                coords = [start_latlon, end_latlon]

            _add_polyline_with_border(fg_rt,
                coords,
                color=cmap(d),
                weight=4,
                opacity=0.9,
                tooltip=(f"<div style='font-size:14px; font-weight:bold'>"
                    f"{season_label} • {label_hr} • Route {rt}"
                         f"<br>start_id: {start_id} → end_id: {end_id}"
                         f"<br>Δ̄={d:.2f} (act̄={r['act_mean']:.2f}, sch̄={r['sch_mean']:.2f})")
            )


        route_groups.append(fg_rt)

    summary = seg_agg[['time_point_id','time_point_order_end',
                       'Lat_x','Long_x','Lat_y','Long_y',
                       'routes',               
                       'n_routes','act_mean','sch_mean','delta']].copy()


    return fg_shared, fg_all, route_groups, summary


In [9]:
def _in_hour_range(series, hour_or_range):
    h = series.astype(float)
    if isinstance(hour_or_range, (tuple, list)) and len(hour_or_range) == 2:
        lo, hi = map(float, hour_or_range)
        if lo > hi:  
            lo, hi = hi, lo
        return (h >= lo) & (h <= hi)      
    else:
        return h == float(hour_or_range)
def _hour_label(hour_or_range):
    if isinstance(hour_or_range, (tuple, list)) and len(hour_or_range) == 2:
        lo, hi = map(float, hour_or_range)
        if lo > hi: lo, hi = hi, lo
        return f"{int(lo):02d}:00–{int(hi):02d}:00"
    return f"{int(float(hour_or_range)):02d}:00"


In [10]:
def build_corridor_map(df_all, season_queries, stops_all,
                       seasons,
                       routes,
                       hour, k_first, k_last,
                       direction, DOW,
                       out_path="corridor_map.html"):

    from branca.colormap import LinearColormap as _LinearColormap
    from shapely.geometry import Point as _Point, LineString as _LineString
    try:
        from shapely.ops import substring as _substring
    except Exception:
        _substring = None  

    # -------- helpers (local) --------
    def _compute_segagg(mapit):

        need = ['route_id','hour','k','time_diff_x','time_diff_y','Lat_x','Long_x','Lat_y','Long_y',
                'time_point_id','time_point_order_end']
        sub = mapit.loc[
            (_in_hour_range(mapit['hour'], hour)) &
            (mapit['k'].between(k_first, k_last)),
            need
        ].copy()
        sub = sub[sub['time_diff_y'] > 0]
        if sub.empty:
            return pd.DataFrame()

        for c in ['Lat_x','Long_x','Lat_y','Long_y']:
            sub[c] = pd.to_numeric(sub[c], errors='coerce')
        sub = sub.dropna(subset=['Lat_x','Long_x','Lat_y','Long_y']).copy()
        if sub.empty:
            return pd.DataFrame()

        sub['delta'] = (sub['time_diff_x'] - sub['time_diff_y']) / sub['time_diff_y']

        per_route = (
            sub.groupby(['route_id','time_point_id','time_point_order_end'], as_index=False)
               .agg(delta=('delta','mean'),
                    act_mean=('time_diff_x','mean'),
                    sch_mean=('time_diff_y','mean'),
                    Lat_x=('Lat_x','mean'),
                    Long_x=('Long_x','mean'),
                    Lat_y=('Lat_y','mean'),
                    Long_y=('Long_y','mean'))
        )

        def _routes_agg(x):
            return tuple(sorted(map(str, pd.Series(x).unique())))

        seg_agg = (
            per_route.groupby(['time_point_id','time_point_order_end'], as_index=False)
                     .agg(delta=('delta','mean'),
                          act_mean=('act_mean','mean'),
                          sch_mean=('sch_mean','mean'),
                          Lat_x=('Lat_x','mean'),
                          Long_x=('Long_x','mean'),
                          Lat_y=('Lat_y','mean'),
                          Long_y=('Long_y','mean'),
                          routes=('route_id', _routes_agg))
        )
        seg_agg['n_routes'] = seg_agg['routes'].apply(len)
        seg_agg.replace([np.inf, -np.inf], np.nan, inplace=True)
        return seg_agg.dropna(subset=['Lat_x','Long_x','Lat_y','Long_y','delta','act_mean','sch_mean'])

    def _slice_route_segment(line, p_start_latlon, p_end_latlon):
        """Slice LineString between nearest points to p_start/p_end (lat/lon in, lat/lon out)."""
        if not isinstance(line, _LineString) or _substring is None:
            return [p_start_latlon, p_end_latlon]
        p_start = _Point(p_start_latlon[1], p_start_latlon[0])
        p_end   = _Point(p_end_latlon[1],   p_end_latlon[0])
        try:
            d1, d2 = line.project(p_start), line.project(p_end)
            if d1 == d2:
                return [p_start_latlon, p_end_latlon]
            if d1 > d2:
                d1, d2 = d2, d1
            seg = _substring(line, d1, d2)
            if seg.is_empty:
                return [p_start_latlon, p_end_latlon]
            coords_ll = [(y, x) for (x, y) in seg.coords]
            # ensure order starts near start
            if _Point(coords_ll[0][1], coords_ll[0][0]).distance(p_start) > \
               _Point(coords_ll[-1][1], coords_ll[-1][0]).distance(p_start):
                coords_ll = list(reversed(coords_ll))
            return coords_ll
        except Exception:
            return [p_start_latlon, p_end_latlon]

    label_hr = _hour_label(hour)

    # -------- 1) compute per-season mapit --------
    by_season = {
        s: build_mapit_for_season_corridor(
               df_all, season_queries, stops_all, s,
               routes=routes, direction=direction, DOW=DOW
           )
        for s in seasons
    }

    # -------- 2) base cmap for single-season layers  --------
    deltas = []
    for s, mi in by_season.items():
        sub = mi.loc[
            (_in_hour_range(mi['hour'], hour)) &
            (mi['k'].between(k_first, k_last)),
            ['route_id','time_diff_x','time_diff_y','time_point_id','time_point_order_end']
        ].copy()
        sub = sub[sub['time_diff_y'] > 0]
        if sub.empty:
            continue
        sub['delta'] = (sub['time_diff_x'] - sub['time_diff_y']) / sub['time_diff_y']
        per_route = (sub.groupby(['route_id','time_point_id','time_point_order_end'], as_index=False)
                       .agg(delta=('delta','mean')))
        route_counts = (per_route.groupby(['time_point_id','time_point_order_end'])['route_id']
                                  .nunique().reset_index(name='n_routes'))
        shared = per_route.merge(route_counts, on=['time_point_id','time_point_order_end'])
        shared = shared[shared['n_routes'] >= MIN_ROUTES]
        vals = shared.groupby(['time_point_id','time_point_order_end'])['delta'].mean().values
        if vals.size:
            deltas.append(vals)

    if deltas:
        arr = np.concatenate(deltas)
        vmin = float(np.quantile(arr, 0.05)); vmax = float(np.quantile(arr, 0.95))
        if not np.isfinite(vmin) or not np.isfinite(vmax) or vmin == vmax:
            vmin, vmax = -1.0, 1.0
    else:
        vmin, vmax = -1.0, 1.0

    cmap = _LinearColormap(["#2c7bb6", "#ffffbf", "#d7191c"], vmin=vmin, vmax=vmax)
    cmap.caption = "Δ runtime = (act − schd)/schd (shared-segment mean)"

    # -------- 3) map centre --------
    any_season = next(iter(by_season.values()))
    lat_center = float(np.nanmean(pd.concat([any_season['Lat_x'], any_season['Lat_y']])))
    lon_center = float(np.nanmean(pd.concat([any_season['Long_x'], any_season['Long_y']])))
    m = folium.Map(location=[lat_center, lon_center], zoom_start=12, tiles="cartodbpositron")

    # -------- 4) per-season layers (existing) --------
    shared_all_layers = []    # dropdown 1
    route_layers_s1   = []    # dropdown 2
    route_layers_s2   = []    # dropdown 3

    summaries = []
    first_shared_shown = True

    for s in seasons:
        fg_shared, fg_all, route_groups, tbl = _corridor_layers_with_route_counts(
            by_season[s], hour, k_first, k_last, cmap, s,
            min_routes=MIN_ROUTES,
            route_geoms=route_geoms  
        )

        if fg_shared is not None:
            fg_shared.overlay = True; fg_shared.control = True
            if first_shared_shown:
                fg_shared.show = True; first_shared_shown = False
            fg_shared.add_to(m); shared_all_layers.append(fg_shared)

        if fg_all is not None:
            fg_all.overlay = True; fg_all.control = True
            fg_all.add_to(m); shared_all_layers.append(fg_all)

        for fg_rt in route_groups:
            fg_rt.overlay = True; fg_rt.control = True
            fg_rt.add_to(m)

        if s == seasons[0]:
            route_layers_s1.extend(route_groups)
        elif len(seasons) > 1 and s == seasons[1]:
            route_layers_s2.extend(route_groups)

        if tbl is not None:
            tbl['season'] = s
            summaries.append(tbl)

    # -------- Two-season comparison overlays --------
    if len(seasons) >= 2:
        s0, s1 = seasons[0], seasons[1]
        seg0 = _compute_segagg(by_season[s0])
        seg1 = _compute_segagg(by_season[s1])

        if not seg0.empty and not seg1.empty:
            # keep segments shared in BOTH seasons
            seg0s = seg0[seg0['n_routes'] >= MIN_ROUTES].copy()
            seg1s = seg1[seg1['n_routes'] >= MIN_ROUTES].copy()

            cmp = seg0s.merge(seg1s,
                              on=['time_point_id','time_point_order_end'],
                              suffixes=('_a','_b'))
            if not cmp.empty:
                # choose coordinates (average of seasons, more robust)
                for c in ['Lat_x','Long_x','Lat_y','Long_y']:
                    cmp[c] = (cmp[f"{c}_a"] + cmp[f"{c}_b"]) / 2.0

                def _choose_route(row):
                    A = set(map(str, row['routes_a']))
                    B = set(map(str, row['routes_b']))
                    inter = sorted(A & B)
                    if inter:
                        return inter[0]
                    uni = sorted(A | B)
                    return uni[0] if uni else None

                cmp['rt_choice'] = cmp.apply(_choose_route, axis=1)

                # compute diffs
                cmp['diff_abs'] = cmp['delta_b'] - cmp['delta_a']
                eps = 1e-6
                denom = np.maximum(np.abs(cmp['delta_a']), eps)
                cmp['diff_pct'] = 100.0 * cmp['diff_abs'] / denom

                # color scales (symmetric around 0 using 5–95% quantiles)
                def _sym_range(v):
                    if v.empty:
                        return (-1.0, 1.0)
                    q1, q2 = np.quantile(v, [0.05, 0.95])
                    maxabs = float(max(abs(q1), abs(q2)))
                    if not np.isfinite(maxabs) or maxabs == 0:
                        maxabs = 1.0
                    return (-maxabs, maxabs)

                vmin_abs, vmax_abs = _sym_range(cmp['diff_abs'].astype(float))
                vmin_pct, vmax_pct = _sym_range(cmp['diff_pct'].astype(float))

                cmap_abs = _LinearColormap(["#2c7bb6", "#ffffbf", "#d7191c"],
                                           vmin=vmin_abs, vmax=vmax_abs)
                cmap_abs.caption = f"Δ change (abs) • {s1} − {s0}"

                cmap_pct = _LinearColormap(["#2c7bb6", "#ffffbf", "#d7191c"],
                                           vmin=vmin_pct, vmax=vmax_pct)
                cmap_pct.caption = f"Δ change (%) • relative to {s0}"

                # A) absolute diff overlay
                fg_diff_abs = folium.FeatureGroup(
                    name=f"Compare {s1} − {s0} • abs(Δ change) • {label_hr}",
                    overlay=True, control=True, show=False
                )
                # B) percent diff overlay
                fg_diff_pct = folium.FeatureGroup(
                    name=f"Compare {s1} vs {s0} • %Δ of Δ • {label_hr}",
                    overlay=True, control=True, show=False
                )

                # draw both overlays using actual route segment geometry
                for _, r in cmp.iterrows():
                    start_latlon = (float(r['Lat_x']), float(r['Long_x']))
                    end_latlon   = (float(r['Lat_y']), float(r['Long_y']))

                    # pick a route geometry
                    coords = [start_latlon, end_latlon]
                    if 'route_geoms' in globals() and route_geoms is not None:
                        line_full = route_geoms.get(str(r['rt_choice']))
                        if isinstance(line_full, _LineString):
                            coords = _slice_route_segment(line_full, start_latlon, end_latlon)

                    # absolute overlay
                    _add_polyline_with_border(
                        fg_diff_abs, coords,
                        color=cmap_abs(float(r['diff_abs'])),
                        weight=6,
                        opacity=0.9,
                        tooltip=(f"<div style='font-size:14px; font-weight:bold'>"
                            f"{s0} vs {s1} • {label_hr}"
                                 f"<br>start_id: {r['time_point_id']} → end_id: {r['time_point_order_end']}"
                                 f"<br>Δ_a={r['delta_a']:.3f}, Δ_b={r['delta_b']:.3f}"
                                 f"<br>abs change = {r['diff_abs']:.3f}")
                    )
                    # percent overlay
                    _add_polyline_with_border(
                        fg_diff_pct, coords,
                        color=cmap_pct(float(r['diff_pct'])),
                        weight=6,
                        opacity=0.9,
                        tooltip=(f"<div style='font-size:14px; font-weight:bold'>"
                            f"{s0} vs {s1} • {label_hr}"
                                 f"<br>start_id: {r['time_point_id']} → end_id: {r['time_point_order_end']}"
                                 f"<br>Δ_a={r['delta_a']:.3f}, Δ_b={r['delta_b']:.3f}"
                                 f"<br>% change = {r['diff_pct']:.1f}%")
                    )

                # add both comparison overlays + legends to map and to first menu
                fg_diff_abs.add_to(m); shared_all_layers.append(fg_diff_abs)
                fg_diff_pct.add_to(m); shared_all_layers.append(fg_diff_pct)
                cmap_abs.add_to(m); cmap_pct.add_to(m)

    # -------- 6) add legends/controls --------
    cmap.add_to(m)

    # THREE independent GroupedLayerControls (first one includes comparison overlays)
    GroupedLayerControl(
        groups={"Shared / All": shared_all_layers},
        exclusive_groups=[],
        collapsed=True,
        position="topright"
    ).add_to(m)

    if route_layers_s1:
        GroupedLayerControl(
            groups={f"Routes – {seasons[0]}": route_layers_s1},
            exclusive_groups=[],
            collapsed=True,
            position="topright"
        ).add_to(m)

    if route_layers_s2:
        GroupedLayerControl(
            groups={f"Routes – {seasons[1]}": route_layers_s2},
            exclusive_groups=[],
            collapsed=True,
            position="topright"
        ).add_to(m)

    # -------- 7) save & return --------
    m.save(out_path)
    summary_df = (pd.concat(summaries, ignore_index=True)
                    .sort_values(['season','time_point_id','time_point_order_end'])
                  if summaries else pd.DataFrame())
    return out_path, summary_df


In [12]:
out_path, table = build_corridor_map(df_all=df_all, 
                                     season_queries=season_queries, 
                                     stops_all=stops_all, 
                                     seasons=('2025-2','2025-1'), 
                                     #routes=['22','23','28'],
                                     #routes=['34'],
                                     routes = all_available_routes['route_id'],
                                     hour = (13,16), 
                                     k_first=1, k_last=333, 
                                     direction='Inbound', 
                                     DOW='Weekday', 
                                     out_path="corridor.html" ) 
print("Saved:", out_path)

Saved: corridor.html
